In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from code.evaluation import (compute_horizon_degradation_metrics,
                             plot_forecast_degradation,
                             plot_selected_horizons,
                             plot_forecast_trajectory,
                             compute_forecast_metrics)

from code.data_utils import clean_actual_data, clean_forecast_data

In [6]:
actual_df = pd.read_csv(r'../../data/processed/actual_load.csv', parse_dates=['timestamp'])
forecast_df = pd.read_csv(r'../../data/processed/forecast_load.csv', parse_dates=['issued_at', 'target_time'])